In [1]:
import sys
from opentree import OT

In [2]:
!wget https://raw.githubusercontent.com/McTavishLab/biodiversity_next/master/example.csv

--2019-12-29 16:26:14--  https://raw.githubusercontent.com/McTavishLab/biodiversity_next/master/example.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.128.133, 151.101.64.133, 151.101.192.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.128.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49361 (48K) [text/plain]
Saving to: ‘example.csv.3’

example.csv.3       100%[===================>]  48.20K  --.-KB/s    in 0.08s   

2019-12-29 16:26:14 (620 KB/s) - ‘example.csv.3’ saved [49361/49361]



In [3]:
!head example.csv

gbifID	datasetKey	occurrenceID	kingdom	phylum	class	order	family	genus	species	infraspecificEpithet	taxonRank	scientificName	verbatimScientificName	verbatimScientificNameAuthorship	countryCode	locality	stateProvince	occurrenceStatus	individualCount	publishingOrgKey	decimalLatitude	decimalLongitude	coordinateUncertaintyInMeters	coordinatePrecision	elevation	elevationAccuracy	depth	depthAccuracy	eventDate	day	month	year	taxonKey	speciesKey	basisOfRecord	institutionCode	collectionCode	catalogNumber	recordNumber	identifiedBy	dateIdentified	license	rightsHolder	recordedBy	typeStatus	establishmentMeans	lastInterpreted	mediaType	issue
2423004790	50c9509d-22c7-4a22-a47d-8c48425ef4a7	https://www.inaturalist.org/observations/32478397	Animalia	Arthropoda	Insecta	Odonata	Libellulidae	Tramea	Tramea lacerata		SPECIES	Tramea lacerata Hagen, 1861	Tramea lacerata		US		California			28eb1a3f-1c15-4a95-931a-4af90ecb574d	37.36422	-120.424003	12.0						2019-09-10T08:39:35Z	10	9	2019	1428475	1428475	HUMAN_OB

In [4]:
filename = "example.csv"
fi = open(filename)
header = fi.readline().split('\t')

In [5]:
#Get indexes for each column in the csv file
col_dict = {}
for i, col in enumerate(header):
    col_dict[col] = i

In [6]:
col_dict

{'gbifID': 0,
 'datasetKey': 1,
 'occurrenceID': 2,
 'kingdom': 3,
 'phylum': 4,
 'class': 5,
 'order': 6,
 'family': 7,
 'genus': 8,
 'species': 9,
 'infraspecificEpithet': 10,
 'taxonRank': 11,
 'scientificName': 12,
 'verbatimScientificName': 13,
 'verbatimScientificNameAuthorship': 14,
 'countryCode': 15,
 'locality': 16,
 'stateProvince': 17,
 'occurrenceStatus': 18,
 'individualCount': 19,
 'publishingOrgKey': 20,
 'decimalLatitude': 21,
 'decimalLongitude': 22,
 'coordinateUncertaintyInMeters': 23,
 'coordinatePrecision': 24,
 'elevation': 25,
 'elevationAccuracy': 26,
 'depth': 27,
 'depthAccuracy': 28,
 'eventDate': 29,
 'day': 30,
 'month': 31,
 'year': 32,
 'taxonKey': 33,
 'speciesKey': 34,
 'basisOfRecord': 35,
 'institutionCode': 36,
 'collectionCode': 37,
 'catalogNumber': 38,
 'recordNumber': 39,
 'identifiedBy': 40,
 'dateIdentified': 41,
 'license': 42,
 'rightsHolder': 43,
 'recordedBy': 44,
 'typeStatus': 45,
 'establishmentMeans': 46,
 'lastInterpreted': 47,
 'medi

In [7]:
match_dict = {}
ott_ids = set()

#Loop through each line in the gbif output
for lin in fi:
    lii = lin.split('\t')
    gb_id = lii[col_dict['taxonKey']]
    sys.stdout.write(".") #progress bar
    sys.stdout.flush()
    if gb_id in match_dict:
        #Skip gb_id's you have already matched
        pass
    else:
        # Do a direct match to gbif id's in the open tree taxnomy
        gbiftax = "gbif:{}".format(int(gb_id))
        res = OT.taxon_info(source_id=gbiftax)
        if res.status_code == 200:
            ott_id = int(res.response_dict['ott_id'])
            match_dict[gb_id] = ott_id
        if res.status_code == 400:
            # If GBIF id isn' found in the open tree taxonomy, search on scientific name
            spp_name = lii[col_dict['verbatimScientificName']]
            res2 = OT.tnrs_match([spp_name])
            if res2.status_code == 200:
                if len(res2.response_dict['results']) > 0:
                    ott_id = int(res2.response_dict['results'][0]['matches'][0]['taxon']['ott_id'])
                    match_dict[gb_id] = ott_id
                    sys.stdout.write("{},{} matched on name to ott id{}\n".format(gbiftax, spp_name, ott_id))
                else:
                    sys.stdout.write("{},{} still NO MATCH\n".format(gbiftax, spp_name))
                    match_dict[gb_id] = None
        ott_ids.add(ott_id)

........................gbif:5229155,Pelecanus erythrorhynchos not matched on ID
gbif:5229155,Pelecanus erythrorhynchos matched on name to ott id316989
....gbif:9088491,Dryobates nuttallii not matched on ID
gbif:9088491,Dryobates nuttallii matched on name to ott id701703
..gbif:5231677,Mimus polyglottos not matched on ID
gbif:5231677,Mimus polyglottos matched on name to ott id571310
......................................gbif:2498167,Anser caerulescens not matched on ID
gbif:2498167,Anser caerulescens matched on name to ott id190878
...........gbif:2498161,Anser rossii not matched on ID
gbif:2498161,Anser rossii matched on name to ott id767830
....................

In [9]:
len(ott_ids)

56

In [10]:
trefile = "names.tre"
#Get the syntehtic tree from OpenTreem and withe the ciattions to a text file.
output = OT.synth_induced_tree(ott_ids=list(ott_ids),  label_format='name')
output.tree.write(path = "names.tre", schema = "newick")
sys.stdout.write("Tree written to {}\n".format(trefile))

Tree written to names.tre


In [11]:
output.tree.as_string(schema="newick")

"(((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((Regulus_calendula)Regulus)mrcaott246ott3599436,(((((((((Sturnus_vulgaris)mrcaott366470ott565813)mrcaott2224ott366470)mrcaott2175ott2224)mrcaott2175ott59905,((((((((Mimus_polyglottos)mrcaott571310ott1000585)mrcaott349284ott571310)mrcaott349284ott817639)mrcaott349284ott556541)mrcaott325207ott349284)mrcaott259082ott325207)mrcaott259082ott259086)mrcaott259082ott474281)mrcaott2175ott259082)mrcaott2175ott968664)mrcaott1566ott2175)mrcaott1566ott496009)mrcaott1566ott3598440)mrcaott246ott1566,(((((((Phainopepla_nitens)Phainopepla)mrcaott33427ott294590)mrcaott33427ott179330)mrcaott5934ott33427)mrcaott5934ott496029)mrcaott5934ott294597)mrcaott5934ott33425)mrcaott246ott5934,(((((((((((((((((((((((Passerculus_sandwichensis)Passerculus)mrcaott292636ott453758)mrcaott261498ott292636)mrcaott261498ott292635)mrcaott261498ott934463)mrcaott261498ott292639)mrcaott6023ott261498,(((((Zonotrichia_leucophrys)mrcaott125079ott265554)mrcaott1250

In [16]:
studies = output.response_dict['supporting_studies']

In [17]:
for study in studies:
    studyid = study.split('@')[0]
    studyres = OT.find_studies(studyid, search_property = 'ot:studyId', verbose="True")
    new_cite = studyres.response_dict.get('matched_studies', None)
    if new_cite:
        print(new_cite[0].get('ot:studyPublicationReference', '') + '\n' + new_cite[0].get('ot:studyPublication', '') + '\n')



Carrano M.T., Benson R.B., & Sampson S.D. 2012. The phylogeny of Tetanurae (Dinosauria: Theropoda). Journal of Systematic Palaeontology, 10(2): 211-300.
http://dx.doi.org/10.1080/14772019.2011.630927

Brusatte S.L., Benton M.J., Desojo J.B., & Langer M.C. 2010. The higher-level phylogeny of Archosauria (Tetrapoda: Diapsida). Journal of Systematic Palaeontology, 8(1): 3-47.
http://dx.doi.org/10.1080/14772010903537732

Andres B.B., Clark J.M., & Xu X. 2012. A new rhamphorhynchid pterosaur from the Upper Jurassic of Xinjiang, China, and the phylogenetic relationships of basal pterosaurs. Journal of Vertebrate Paleontology, 30(1): 163-187.
http://dx.doi.org/10.1080/02724630903409220

Zanno L.E. 2010. A taxonomic and phylogenetic re-evaluation of Therizinosauria (Dinosauria: Maniraptora). Journal of Systematic Palaeontology, 8(4): 37-41.
http://dx.doi.org/10.1080/14772019.2010.488045

Dongyu H., Lianhai H., Lijun Z., & Xing X. 2010. A pre-Archaeopteryx troodontid theropod from China with lo

ERROR:root:Error in POST to https://api.opentreeoflife.org/v3/studies/find_studies
Traceback (most recent call last):
  File "/home/ejmctavish/projects/otapi/python-opentree/opentree/ws_wrapper.py", line 166, in _call_api
    call_obj = self._http_request(url, http_method, data=data, headers=headers)
  File "/home/ejmctavish/projects/otapi/python-opentree/opentree/ws_wrapper.py", line 231, in _http_request
    allow_redirects=True)
  File "/home/ejmctavish/projects/otapi/python-opentree/venv-jupyentree/lib/python3.7/site-packages/requests/api.py", line 60, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/ejmctavish/projects/otapi/python-opentree/venv-jupyentree/lib/python3.7/site-packages/requests/sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/ejmctavish/projects/otapi/python-opentree/venv-jupyentree/lib/python3.7/site-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs

Jønsson, Knud Andreas, Pierre-Henri Fabre, Jonathan D. Kennedy, Ben G. Holt, Michael K. Borregaard, Carsten Rahbek, Jon Fjeldså. 2016. A supermatrix phylogeny of corvoid passerine birds (Aves: Corvides). Molecular Phylogenetics and Evolution 94: 87-94.
http://dx.doi.org/10.1016/j.ympev.2015.08.020



KeyboardInterrupt: 

In [18]:
studyid

'ot_504'

In [19]:
new_cite

[{'ot:studyId': 'ot_520',
  'ot:focalCladeOTTTaxonName': 'Corvoidea',
  'ot:dataDeposit': 'http://dx.doi.org/10.5061/dryad.v0np1',
  'ot:focalClade': 635217,
  'ot:studyPublicationReference': 'Jønsson, Knud Andreas, Pierre-Henri Fabre, Jonathan D. Kennedy, Ben G. Holt, Michael K. Borregaard, Carsten Rahbek, Jon Fjeldså. 2016. A supermatrix phylogeny of corvoid passerine birds (Aves: Corvides). Molecular Phylogenetics and Evolution 94: 87-94.',
  'ot:studyPublication': 'http://dx.doi.org/10.1016/j.ympev.2015.08.020',
  'ot:studyYear': 2016,
  'ot:curatorName': ['Joseph W. Brown', 'Jonathan A Rees', 'Karen Cranston'],
  'ot:tag': []}]

In [20]:
from opentree.helpers import get_citations

In [23]:
get_citations(studies)

AttributeError: 'str' object has no attribute 'append'

In [48]:
output